In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import neurokit2 as nk

In [ ]:
def format_x_axis(ax):
    years = mdates.YearLocator()   # every year
    months = mdates.MonthLocator()  # every month
    yearsFmt = mdates.DateFormatter('%Y')
    ax.xaxis.set_major_locator(years)
    ax.xaxis.set_major_formatter(yearsFmt)
    ax.xaxis.set_minor_locator(months)
    ax.tick_params(axis='x', labelrotation=45)

# ARMA and friends

The models we describe in this notebook can be called *classical models*, as they have been worked out at the beginnig of the previous century and therefore have a longer tradition than more recent statistical models.

A time-series process can be either stationary or non-stationary. Stationarity is defined by the following three characteristics:

1. Finite *variation*
2. Constant *mean*
3. Constant *variation*

Many time-series exhibit trends and seasonality, while many others assume stationarity. If a time-series is stationary, its mean and standard deviation stays constant over time. This implies that the time-series has no trend and no cyclic variability. Therefore the removal of irregular components, trends, and seasonal fluctuations is an intrinsic aspect of applying classical models. The models then forecast what's left after this removal.

## Dickey-Fuller test
To check for stationarity, we can use the [Dickey-Fuller Test](https://en.wikipedia.org/wiki/Dickey%E2%80%93Fuller_test). This test specifically checks for the presence of a unit root in the time series, which would indicate non-stationarity. A unit root means the series has a trend and is not stationary.

In [ ]:
ecg_data = nk.ecg_simulate(duration=10)
nk.signal_plot(ecg_data, figsize=(12,6))

In [ ]:
result = adfuller(ecg_data)

# Extract results
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
print(f'Critical Values: {result[4]}')

We have a very small `p-value`, which means that our data is stationary. That resonates with the three characteristics of stationary data. Lets' do the same for more random data:

In [ ]:
np.random.seed(42)
random_walk = np.random.normal(loc=0, scale=1, size=1000).cumsum()
random_walk_series = pd.Series(random_walk)

plt.figure(figsize=(10, 6))
plt.plot(random_walk_series, color='purple')
plt.title('Random Walk (Non-Stationary Data)')
plt.xlabel('Time')
plt.ylabel('Value')
plt.show()


In [ ]:
result = adfuller(random_walk_series)

print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
print(f'Critical Values: {result[4]}')

The ADF statistic is higher than the critical values at the 1%, 5%, and 10% levels. Since the p-value is much greater than 0.05, we fail to reject the null hypothesis that the data has a unit root. Therefore, the data is non-stationary, which is expected since we generated it as a random walk (a typical non-stationary series)

## Moving Average

The *Moving Average* (MA), the unweighted mean over a period of $k$ points, is defined as follows:

$$
MA = \frac{1}{k}\Sigma_{i=1}^{k}x_i
$$ 

where $x_i$ is the observed time-series.

The MA can be used to smooth out a time-series, thereby removing noise and periodic fluctuations that occur in the short term, effectively working as a low-pass filter. We have seen this in our [`demo_univariate_modelling`-notebook](demo_univariate_modelling.ipynb').

In [ ]:
df = pd.read_csv('../data/btc.csv', names=['date','value'], skiprows=1, parse_dates=[0])
df['avg'] = df['value'].rolling(30).mean()
#df.info()
df.head()

In [ ]:
fig, ax = plt.subplots()
format_x_axis(ax)
ax.plot(df['date'], df['value'], label='value')
ax.plot(df['date'], df['avg'], label='rolling mean')

ax.legend()
plt.show()

We can also use MA to forecast into the future as well. The time-series is a linear regression of the current value of the series against observed values. For this to work, we can use the [*Autoregressive Model*](https://en.wikipedia.org/wiki/Autoregressive_model), which regresses the variable on its own lagged values: the current value of the value is driven by immediately preceding values using a linear combination. This way we get an *autoregressive–moving-average* or ARMA-model. 

The Autoregressive (AR) part of the ARMA model uses the relationship between an observation and a number of lagged (previous) observations to predict future values. The Moving Average (MA) part of the ARMA model uses the dependency between an observation and a residual error from a moving average model applied to lagged observations.

## ARMA model
The ARMA-model consists of two types of lagged values, one for the autoregressive component and the other fot he moving average component. Therefore, we write $ARMA(p,q)$, where $p$ indicates the order of the autoregression, and $q$ the order of the moving average. Thus

$$
ARMA(p,q):x_t = c + \epsilon_t + \Sigma_{i=1}^{p}\phi_ix_{t-i} + \Sigma_{i=0}^{q}\Phi_i\epsilon_{t-i}
$$

where  $c$ is a constant, $\phi_i$ and $\Phi_i$ are *model parameters* and $\epsilon_t$ represents noise. ARMA assumes that the series is stationary. 

To demonstrate ARMA in statsmodels, we need to use the [`ARIMA`-class](https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima.model.ARIMA.html), with the differencing-parameter set to zero (0). We'll get to this in a moment. 

In [ ]:
# First we define a method to plot our results.

def plot_model_results(ax, data, models=[], names=[], colors=[], steps=30, color='red'):
    assert len(models) == len(names)
    assert len(models) == len(colors)
    ax.plot(data, label='Observed Data', color='blue')
    ax.axvline(n, color='gray', linestyle='dotted', label='Forecast Start')
    
    for i, model in enumerate(models):
        forecast = model.get_forecast(steps=steps)
        preds = forecast.predicted_mean    
        conf = forecast.conf_int()
        idx = np.arange(n, n + steps)
        
        #plot the forecasts:
        ax.plot(idx, preds, label=f"{names[i]} Forecast", color=colors[i], linestyle='dashed')
        # Plot the confidence Intervals
        ax.fill_between(idx, conf[:, 0], conf[:, 1], color=colors[i], alpha=0.2)
    
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')

In [ ]:
# Let's generate some seasonal data (sinusoidal with noise)
np.random.seed(42)
seasonality = 20 # 20-period seasonality
n = 100
x = np.arange(n)
seasonal_data = 10 * np.sin(2 * np.pi * x / seasonality) + np.random.normal(0, 2, size=n)  

In [ ]:
arma_model = sm.tsa.ARIMA(seasonal_data, order=(2, 0, 1))
arma_results = arma_model.fit()
arma_results.summary()

In [ ]:
arma_results.params[-1]

In [ ]:
fig,ax = plt.subplots(figsize=(12,6))
plot_model_results(ax, seasonal_data, models=[arma_results], names=['ARMA'], colors=['red'])
plt.legend(loc='best')
plt.show()

## The ARIMA model

A generalisation of the ARMA model is the Autoregressive *Integrated* Moving Average model. It is especially used for non-stationary data, for it includes data pre-processing step to make the data more stationary. This is done by replacing values by subtracting the immediate past values, a transformation called *differencing*. The model integration is parametrized by $d$, which is the number of times differences have been taken between current and previous values. So, an ARIMA model is capable of capturing trands but not seasonality.

Note that in te cell below we use the same class as above, only with different values for the `order`-parameter.

In [ ]:
arima_model = sm.tsa.ARIMA(seasonal_data, order=(2, 1, 1))
arima_results = arima_model.fit()
arima_results.summary()


In [ ]:
fig,ax = plt.subplots(figsize=(12,6))
plot_model_results(ax, seasonal_data, models=[arma_results, arima_results], names=['ARMA', 'ARIMA'], colors=['red','green'])
plt.legend()
plt.show()

## The SARIMA model

To account for the seasonality of the data, we have another generalisation of the ARMA model, called SARIMA (we didn't make this up). SARIMO models are usually stated as $ARIMA(p,d,q)(P,D,Q)m$, where

- $m$ denotes the number of periods in a season
- $P,D,Q$ parametrize the autoregressive, integration, and moving average component of the *sesonal part*
- $p,d,q$ are the parameters of the ARIMA-part, that we discussed above.

For SARIMA, we have another class that we can use, [`SARIMAX`](https://www.statsmodels.org/stable/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html#statsmodels.tsa.statespace.sarimax.SARIMAX).

In [ ]:
sarima_model = sm.tsa.SARIMAX(seasonal_data, order=(2, 1, 1), seasonal_order=(1, 1, 1, 20))
sarima_results = sarima_model.fit()
sarima_results.summary()

In [ ]:
fig,ax = plt.subplots(figsize=(12,6))
plot_model_results(ax, seasonal_data, 
                   models=[arma_results, arima_results, sarima_results],
                   names=['ARMA', 'ARIMA', 'SARIMA'],
                   colors=['red', 'green', 'cyan'])
plt.legend()
plt.show()

## Statistical interpretation

From the plot above, you can conclude that for this seasonal data, SARIMA is by far the better predictor. Let's compare the three models numerically.

In [ ]:
model_comparison = pd.DataFrame({
    "ARMA": [arma_results.aic, arma_results.bic, arma_results.params[-1], arma_results.pvalues.max()],
    "ARIMA": [arima_results.aic, arima_results.bic, arima_results.params[-1], arima_results.pvalues.max()],
    "SARIMA": [sarima_results.aic, sarima_results.bic, sarima_results.params[-1], sarima_results.pvalues.max()]
}, index=["AIC", "BIC", r"$\sigma^2$", "Max p-value"])

# Display the table
model_comparison

The relatively Low AIC and BIC suggest that the SARIMA-model fits well. Since its AIC and BIC values are the lowest, this model explains the data better than the others. A lower AIC/BIC usually means a better balance between fit and complexity. However, the high $\sigma^2$ (3.3) suggests more residual variability. Finally, the high max p-value (0.99) is a red flag: it shows that at least on coefficient is statistically not significant. We can see this already in the summary above, actually: all the coefficients are way higher than 0.05.

As an exercise, let's continue working on this last model, removing coefficients to see its AIC/BIC values improve (get lower). The coefficient with the highest p-value is `ma.S.L20`. Let's decrease this value:

In [ ]:
sarima2_model = sm.tsa.SARIMAX(seasonal_data, order=(2, 1, 1), seasonal_order=(1, 1, 0, 20))
sarima2_results = sarima_model.fit()
sarima2_results.summary()

As you see, this has no direct effect on the model. More research is necessary to see if we can improve on it; but that's left as an exercise to the reader 😎.